In [1]:
from dotenv import dotenv_values
import requests

from helpers.google_interactor import GoogleInteractor
from helpers.html_parser import HTMLParser 
from helpers.storage_manager import StorageManager
from helpers.refiner_model import RefinerModel
from helpers.improver_model import ImproverModel


# Load environment variables
config = dotenv_values("../.env")
google_api_key = config['GOOGLE_API_KEY']
search_engine_id = config['SEARCH_ENGINE_ID']

# Instantiate helpers
google_interactor = GoogleInteractor(api_key=google_api_key, search_engine_id=search_engine_id)
storage_manager = StorageManager("../data/")    
website_refiner = RefinerModel(api_key=config['OPEN_AI_API_KEY'])
corpus_refiner = RefinerModel(api_key=config['OPEN_AI_API_KEY'])
improver_model = ImproverModel(api_key=config['OPEN_AI_API_KEY'])


# Load the clean corpus text
with open("../data/clean_content.txt") as f:
    corpus_text = f.read()

# Number of links to consider
K = 5

# Declare trial number
trial_number = 2

# Select a query to use for the current trial
query = "What products are offered by Southern Farm Bureau Life Insurance Company?"

# Submit the query to Google
search_results = google_interactor.search_google(query=query, num_results=10)
    
# Store the response
storage_manager.save_to_folder("InitialGoogleResponses", search_results, f"{trial_number}.json")


"""
Get Raw Text from the HTML associated with the Google Links
"""

# Retrieve the stored response
search_results = storage_manager.load_from_folder("InitialGoogleResponses", f"{trial_number}.json")

raw_text_from_html = []
items = search_results['items']

# For each item of raw text that was stored ...
for item in search_results['items'][:K]:
    link = item['link']

    # Fet the HTML associated with the link
    response = requests.get(link)

    # Get the raw text associated with the HTML
    parsed_text = HTMLParser.parse_html(response.content)

    # Store the raw text from the HTML
    raw_text_from_html.append({"link" : link, "raw_text" : parsed_text})
    
storage_manager.save_to_folder("RawTextFromHTML", raw_text_from_html, f"{trial_number}.json")


"""
Get the refined text from the raw text
"""

raw_text_from_html = storage_manager.load_from_folder("RawTextFromHTML", f"{trial_number}.json")

website_refiner_responses = []
# For each element of raw text that was stored ...
for current_item in raw_text_from_html:
    
    link = current_item['link']
    raw_text = current_item['raw_text']   

    
    response_from_website_refiner = website_refiner.refine_text(raw_text=raw_text, query=query)
    
    if response_from_website_refiner:
        response_from_website_refiner['link'] = link
        website_refiner_responses.append(response_from_website_refiner)
    else:
        print("Failed to refine text.")

storage_manager.save_to_folder("WebsiteRefinerModelResponses", website_refiner_responses, f"{trial_number}.json")


"""
Filter the corpus to information that may be relevant to the query
"""

response_from_corpus_refiner = corpus_refiner.refine_text(raw_text=corpus_text, query=query)
storage_manager.save_to_folder("CorpusRefinerModelResponses", response_from_corpus_refiner, f"{trial_number}.json")

Data saved to ../data/InitialGoogleResponses/2.json
Data loaded from ../data/InitialGoogleResponses/2.json
Data saved to ../data/RawTextFromHTML/2.json
Data loaded from ../data/RawTextFromHTML/2.json
Data saved to ../data/WebsiteRefinerModelResponses/2.json
Data saved to ../data/CorpusRefinerModelResponses/2.json


In [2]:
# Get the refined text from the website
website_refiner_responses = storage_manager.load_from_folder("WebsiteRefinerModelResponses", f"{trial_number}.json")
sfblic_response = [response for response in website_refiner_responses if response['link'] == 'https://www.sfbli.com/'][0]
refined_text_from_website = sfblic_response['choices'][0]['message']['content']

# Get the refined text from the corpus
response_from_corpus_refiner = storage_manager.load_from_folder("CorpusRefinerModelResponses", f"{trial_number}.json")
refined_text_from_corpus = response_from_corpus_refiner['choices'][0]['message']['content']
refined_text_from_corpus

Data loaded from ../data/WebsiteRefinerModelResponses/2.json
Data loaded from ../data/CorpusRefinerModelResponses/2.json


'Southern Farm Bureau Life Insurance Company offers the following products:\n\n1. **Life Insurance Products:**\n   - **Permanent Life Insurance:** Whole life, single premium, modified premium, and adjustable premium options.\n   - **Term Life Insurance:** Coverage available in 10, 20, and 30-year terms.\n\n2. **Financial Planning Services:**\n   - Estate and retirement planning.\n   - Tax-qualified and non-qualified retirement plans.\n   - Banking services, including certificates of deposit, money market accounts, health savings accounts, and loans.\n\n3. **Additional Services:**\n   - Customized policies for wealth transfer, debt repayment, and daily living costs. \n\nFor more details, visit their website at [sfbli.com](https://www.sfbli.com).'

In [3]:
# Generate improved text
improved_text = improver_model.improve_text(
    query=query,
    refined_text_from_website=refined_text_from_website,
    refined_text_from_corpus=refined_text_from_corpus
)

if improved_text:
    print("Improved Text generated...")
    
else:
    print("Failed to improve text.")

Improved Text generated...


In [4]:
print(improved_text['choices'][0]['message']['content'])

Southern Farm Bureau Life Insurance Company offers a range of products and services, including:

1. **Life Insurance Products:**
   - **Permanent Life Insurance:** Options include whole life, single premium, modified premium, and adjustable premium policies.
   - **Term Life Insurance:** Coverage is available in 10, 20, and 30-year terms.

2. **Financial Planning Services:**
   - Estate and retirement planning.
   - Tax-qualified and non-qualified retirement plans.
   - Banking services, such as certificates of deposit, money market accounts, health savings accounts, and loans.

3. **Additional Services:**
   - Customized policies that cater to wealth transfer, debt repayment, and covering daily living costs.

For more details, please visit their website at [sfbli.com](https://www.sfbli.com).
